# **SDGym - Synthetic Data Gym Metrics Evaluation**

In [1]:
import os
import sys
import platform
import timeit
import numpy as np
import pandas as pd
OS = platform.system()   # Operating System 

#**Data Download**

In [2]:
os.system('git clone https://github.com/istat-methodology/Synthetic_Data.git')

0

In [3]:
os.system('pip install sdgym')
os.system('pip install matplotlib==3.1.3')

0

In [4]:
from sdv.evaluation import evaluate
from sdv.metrics.tabular import CSTest, KSTest                                      # Statistical Metrics
from sdv.metrics.tabular import BNLikelihood, BNLogLikelihood, GMLogLikelihood      # Likelihood Metrics¶
from sdv.metrics.tabular import LogisticDetection, SVCDetection                     # Detection Metrics
from sdv.metrics.tabular import MulticlassDecisionTreeClassifier                    # Machine Learning Efficacy Metrics¶
from sdv.metrics.tabular import NumericalLR 

In [5]:
pd.set_option('display.max_columns', 500) 
pd.set_option('display.max_rows', 500) 

In [6]:
orig_data_path = './Synthetic_Data/Datasets/syntetic_telephony.xlsx'
synth_data_path = './Synthetic_Data/Output/Telephony Synt Results/telephony_synth_data_generated_by_method_copulagantotal_time_2863.58_score_0.45.xlsx'

In [7]:
def explore_data(data): 
  print("\nHead of Data: \n", data.head())
  print("\nTail of Data: \n", data.tail())
  print("\nShape of Data: ", data.shape)
  print("\nInformation about Data: \n")
  try: 
    data.info()
  except: 
    pass
  print("\nTypes of Data attributes: \n")
  try: 
    data.dtypes
  except: 
    pass
  print("\nSummary of all numerical fields in the dataset: \n")
  try: 
    data.describe(include = [np.number])
  except: 
    pass
  print("\nSummary of all categorical fields in the dataset: \n")
  try: 
    data.describe(include = ['O'])
  except: 
    pass
  print("\nLoop Through Each Column and Check for nulls: \n")
  try: 
    for i in range(len(data.columns)):
        print(data.columns[i] + ": " + str(data[data.columns[i]].isna().sum()))
  except: 
    pass

def data_download(file_to_download, gdrive_code, OS, uncompress = True):
  if not os.path.exists(file_to_download):
    os.system('gdown --id "'+gdrive_code+'" --output '+file_to_download)
    if OS == "Linux" and uncompress:
        os.system('unzip -o -n "./'+file_to_download+'" -d "./"')
    return True
  else: 
    return None

In [8]:
xl_file = pd.ExcelFile(orig_data_path)
dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
orig_data = dfs['Sheet1']

In [9]:
# Renaming Columns into English
orig_data.rename(columns = {'COD_CELLA_CHIAMATA':'CELL_CALL_CODE', 
                      'DATA_CHIAMATA':'CALL_DATE',
                      'CHIAVE_NUM_CHIAMANTE':'NUM_CALLER_KEY',
                      'ORA_MIN_CHIAMATA':'TIME_MIN_CALL'}, inplace = True)

# Casting Datatypes  
orig_data.CELL_CALL_CODE=orig_data.CELL_CALL_CODE.astype("int64").astype("str")

orig_data.CALL_DATE=pd.to_datetime(orig_data.CALL_DATE, format='%Y%m%d')
orig_data.NUM_CALLER_KEY=orig_data.NUM_CALLER_KEY.astype("int64").astype("str")

orig_data.TIME_MIN_CALL=orig_data.TIME_MIN_CALL.astype("str").str.pad(width=6, side='left', fillchar='0')
orig_data.TIME_MIN_CALL =pd.to_datetime(orig_data.TIME_MIN_CALL , format='%H%M%S').dt.time.astype("str")
orig_data.TIME_MIN_CALL =pd.to_datetime(orig_data.TIME_MIN_CALL)

explore_data(orig_data)


Head of Data: 
       NUM_CALLER_KEY  CALL_DATE       TIME_MIN_CALL CELL_CALL_CODE
0  14615194667453690 2017-01-22 2022-05-20 15:49:53              0
1  14615194667453690 2017-01-29 2022-05-20 12:40:34              1
2  14615194667453690 2017-01-29 2022-05-20 12:41:19              1
3  14615194667453690 2017-01-22 2022-05-20 15:51:02              1
4  14615194667453690 2017-01-22 2022-05-20 15:51:38              1

Tail of Data: 
          NUM_CALLER_KEY  CALL_DATE       TIME_MIN_CALL CELL_CALL_CODE
9995  14615368752090240 2017-01-12 2022-05-20 12:09:10            272
9996  14615368752090240 2017-02-12 2022-05-20 18:26:01            272
9997   1508338528891430 2017-01-04 2022-05-20 09:26:27            272
9998  14615368752090240 2017-02-10 2022-05-20 15:11:58            272
9999  14615337422429130 2017-01-30 2022-05-20 09:09:03            272

Shape of Data:  (10000, 4)

Information about Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (

In [19]:
orig_data.to_excel('./'+os.path.basename(orig_data_path).rsplit('.', 1)[0]+'_export.xlsx')
orig_data.to_csv('./'+os.path.basename(orig_data_path).rsplit('.', 1)[0]+'_export.csv')

In [20]:
xl_file = pd.ExcelFile(synth_data_path)
dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
synth_data = dfs['Sheet1']

In [21]:
# Renaming Columns into English
synth_data.rename(columns = {'COD_CELLA_CHIAMATA':'CELL_CALL_CODE', 
                      'DATA_CHIAMATA':'CALL_DATE',
                      'CHIAVE_NUM_CHIAMANTE':'NUM_CALLER_KEY',
                      'ORA_MIN_CHIAMATA':'TIME_MIN_CALL'}, inplace = True)

# Removing F character from Dataframe
synth_data["CELL_CALL_CODE"] = synth_data["CELL_CALL_CODE"].str.replace("F","")
synth_data["NUM_CALLER_KEY"] = synth_data["NUM_CALLER_KEY"].str.replace("F","")

# Casting Datatypes  
synth_data.CELL_CALL_CODE=synth_data.CELL_CALL_CODE.astype("int64").astype("str")

synth_data.CALL_DATE=pd.to_datetime(synth_data.CALL_DATE, format='%Y%m%d')
synth_data.NUM_CALLER_KEY=synth_data.NUM_CALLER_KEY.astype("int64").astype("str")

synth_data.TIME_MIN_CALL=synth_data.TIME_MIN_CALL.astype("str").str.pad(width=6, side='left', fillchar='0')
synth_data.TIME_MIN_CALL =pd.to_datetime(synth_data.TIME_MIN_CALL).dt.time.astype("str")
synth_data.TIME_MIN_CALL =pd.to_datetime(synth_data.TIME_MIN_CALL)

synth_data.drop("Unnamed: 0", axis=1, inplace=True)

explore_data(synth_data)


Head of Data: 
       NUM_CALLER_KEY  CALL_DATE       TIME_MIN_CALL CELL_CALL_CODE
0  14615366112382500 2017-01-30 2022-05-20 15:54:19            258
1  14615362915496210 2017-01-14 2022-05-20 11:54:30            781
2  14615430050428180 2017-01-20 2022-05-20 21:25:36             18
3  14615339806425970 2017-01-09 2022-05-20 17:36:41            311
4  14615390824437770 2017-01-06 2022-05-20 13:20:43            565

Tail of Data: 
          NUM_CALLER_KEY  CALL_DATE       TIME_MIN_CALL CELL_CALL_CODE
9995  14615362915496210 2017-01-04 2022-05-20 12:00:18            107
9996  14615224339316630 2017-01-05 2022-05-20 08:12:24            231
9997  14615228723636630 2017-01-02 2022-05-20 16:51:31              9
9998  14615362915496210 2017-01-25 2022-05-20 21:06:11            644
9999  14615420085225040 2017-02-06 2022-05-20 15:19:53            779

Shape of Data:  (10000, 4)

Information about Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (

In [22]:
synth_data.to_excel('./'+os.path.basename(synth_data_path).rsplit('.', 1)[0]+'_export.xlsx')
synth_data.to_csv('./'+os.path.basename(synth_data_path).rsplit('.', 1)[0]+'_export.csv')

#**Model Evaluation**

#**Statistical Metrics**

The metrics of this family compare the tables by running different types of statistical tests on them. In the most simple scenario, these metrics compare individual columns from the real table with the corresponding column from the synthetic table, and at the end report the average outcome from the test.

**sdv.metrics.tabular.KSTest:** This metric uses the two-sample Kolmogorov–Smirnov test to compare the distributions of continuous columns using the empirical CDF. The output for each column is 1 minus the KS Test D statistic, which indicates the maximum distance between the expected CDF and the observed CDF values.

**sdv.metrics.tabular.CSTest:** This metric uses the Chi-Squared test to compare the distributions of two discrete columns. The output for each column is the CSTest p-value, which indicates the probability of the two columns having been sampled from the same distribution.


In [23]:
print(CSTest.compute(orig_data, synth_data))

1.0


**Note:** Inverted Kolmogorov-Smirnov not working because there are not numerical fields within the dataset

In [24]:
start_test_time = timeit.default_timer()
results = evaluate(synth_data, orig_data, metrics=['CSTest', 'KSTest'], aggregate=False)
print("Statistical Metrics Test - Elapsed Time = ", timeit.default_timer() - start_test_time)

Statistical Metrics Test - Elapsed Time =  0.03435272399997302


In [25]:
print(results)

   metric                                     name  raw_score  \
0  CSTest                              Chi-Squared        1.0   
1  KSTest  Inverted Kolmogorov-Smirnov D statistic        NaN   

   normalized_score  min_value  max_value      goal  \
0               1.0        0.0        1.0  MAXIMIZE   
1               NaN        0.0        1.0  MAXIMIZE   

                                        error  
0                                        None  
1  Cannot find fields of types ('numerical',)  


In [26]:
results

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,1.0,1.0,0.0,1.0,MAXIMIZE,None
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('numerical',)"


#**Likelihood Metrics**

The metrics of this family compare the tables by fitting the real data to a probabilistic model and afterwards compute the likelihood of the synthetic data belonging to the learned distribution.

**sdv.metrics.tabular.BNLikelihood:** This metric fits a BayesianNetwork to the real data and then evaluates the average likelihood of the rows from the synthetic data on it.

**sdv.metrics.tabular.BNLogLikelihood:** This metric fits a BayesianNetwork to the real data and then evaluates the average log likelihood of the rows from the synthetic data on it.

**sdv.metrics.tabular.GMLogLikelihood:** This metric fits multiple GaussianMixture models to the real data and then evaluates the average log likelihood of the synthetic data on them.

**Note:** GaussianMixture models not working because there are not numerical fields within the dataset

In [27]:
start_test_time = timeit.default_timer()
results = BNLikelihood.compute(orig_data.fillna(0), synth_data.fillna(0))
print("Likelihood Metrics Test n.1 - Elapsed Time = ", timeit.default_timer() - start_test_time)

Likelihood Metrics Test n.1 - Elapsed Time =  96.64920306099998


In [18]:
type(results)

numpy.float64

In [19]:
results

0.00013183

In [20]:
start_test_time = timeit.default_timer()
results = BNLogLikelihood.compute(orig_data.fillna(0), synth_data.fillna(0))
print("Likelihood Metrics Test n.2 - Elapsed Time = ", timeit.default_timer() - start_test_time)

Likelihood Metrics Test n.2 - Elapsed Time =  62.50358864300051


In [21]:
type(results)

numpy.float64

In [22]:
results

-17.415473543655498

#**Detection Metrics**

The metrics of this family evaluate how hard it is to distinguish the synthetic data from the real data by using a Machine Learning model. To do this, the metrics will shuffle the real data and synthetic data together with flags indicating whether the data is real or synthetic, and then cross validate a Machine Learning model that tries to predict this flag. The output of the metrics will be the 1 minus the average ROC AUC score across all the cross validation splits.

Such metrics are:

**sdv.metrics.tabular.LogisticDetection**: Detection metric based on a LogisticRegression classifier from scikit-learn.

**sdv.metrics.tabular.SVCDetection:** Detection metric based on a SVC classifier from scikit-learn.

In [23]:
start_test_time = timeit.default_timer()
results = LogisticDetection.compute(orig_data, synth_data)
print("Detection Metrics Test n.1 - Elapsed Time = ", timeit.default_timer() - start_test_time)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning

Detection Metrics Test n.1 - Elapsed Time =  11.863354027000241


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [24]:
results

0.6160828732872541

In [ ]:
start_test_time = timeit.default_timer()
results = SVCDetection.compute(orig_data, synth_data)
print("Detection Metrics Test n.2 - Elapsed Time = ", timeit.default_timer() - start_test_time)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [ ]:
results

#**Machine Learning Efficacy Metrics**

This family of metrics will evaluate whether it is possible to replace the real data with synthetic data in order to solve a Machine Learning Problem by learning a Machine Learning model on the synthetic data and then evaluating the score which it obtains when evaluated on the real data.

**Note:** Since this metrics will be evaluated by trying to solve a Machine Learning problem, they can only be used on datasets that contain a target column that needs or can be predicted using the rest of the data, and the scores obtained by the metrics will be inversely proportional to how hard that Machine Problem is.

The metrics on this family are organized by Machine Learning problem type and model.

    Binary Classification Metrics:

        BinaryDecisionTreeClassifier

        BinaryAdaBoostClassifier

        BinaryLogisticRegression

        BinaryMLPClassifier

    Multiclass Classification Metrics:

        MulticlassDecisionTreeClassifier

        MulticlassMLPClassifier

    Regression Metrics:

        LinearRegression

        MLPRegressor

In order to run these metrics we will need to select a column from our dataset which we will use as the target for the prediction problem. 

In [ ]:
MulticlassDecisionTreeClassifier.compute(orig_data, synth_data, target='mba_spec')

#**Privacy Metrics**

This family of metrics measures the privacy of a synthetic dataset by positing the question: given the synthetic data, can an attacker predict sensitive attributes in the real dataset? These models accomplish this by fitting an adversarial attacker model on the synthetic data to predict sensitive attributes from “key” attributes and then evaluating its accuracy on the real data.

The metrics on this family are organized according to the data type they take as input:

    Categorical metrics:

        sdv.metrics.tabular.CategoricalCAP

        sdv.metrics.tabular.CategoricalZeroCAP

        sdv.metrics.tabular.CategoricalGeneralizedCAP

        sdv.metrics.tabular.CategoricalKNN

        sdv.metrics.tabular.CategoricalNB

        sdv.metrics.tabular.CategoricalRF

        sdv.metrics.tabular.CategoricalEnsemble

    Numerical metrics:

        sdv.metrics.tabular.NumericalMLP

        sdv.metrics.tabular.NumericalLR

        sdv.metrics.tabular.NumericalSVR

        sdv.metrics.tabular.NumericalRadiusNearestNeighbor

In addition to the real and synthetic data, these metrics also require two additional inputs, sensitive_fields which is a list of columns considered private and key_fields which are the columns that will be used to try to predict the sensitive ones.

Using the demo data set, one possible example is to use:

    salary as a sensitive column, which is the column we are measuring ahow private it is

    second_perc, mba_perc and degree_perc as the key columns, which will be used by the adversarial attacker to predict the sensitive column

Notice that as all the involved columns are numerical, we need to apply a numerical privacy metric. Conversely, if all of the columns are categorical, we need to use a categorical privacy metric. Currently, the privacy metrics do not support mixed data types.

**Note:**These metrics do not accept missing data, so we will replace all the missing values with a 0 before executing them.

In [ ]:
if OS!="Linux" or colab_active is True:
  NumericalLR.compute(
          data.fillna(0), 
          synthetic_data.fillna(0),
              key_fields=['second_perc', 'mba_perc', 'degree_perc'],
              sensitive_fields=['salary']
          )

#**Output Visual Analysis**

In [ ]:
if OS!="Linux" or colab_active is True and dataset is 'telephony':
  outputData=pd.read_excel("/content/telephony_synth_data_generated_by_method_copulagantotal_time_3309.26_score_0.429.xlsx")

  pd.DataFrame(data.CHIAVE_NUM_CHIAMANTE.value_counts()).hist(bins=20)
  plt.show()

  pd.DataFrame(outputData.CHIAVE_NUM_CHIAMANTE.value_counts()).hist(bins=20)
  plt.show()

  A=set(data.CHIAVE_NUM_CHIAMANTE.astype("str"))
  B=set(outputData.CHIAVE_NUM_CHIAMANTE.astype("str"))
  print(B.difference(A))

  print(data.info())


  pd.DataFrame(data.COD_CELLA_CHIAMATA.value_counts()).hist(bins=20)
  plt.show()

  pd.DataFrame(outputData.COD_CELLA_CHIAMATA.value_counts()).hist(bins=20)
  plt.show()

  outputData.plot.scatter("COD_CELLA_CHIAMATA","DATA_CHIAMATA")
  plt.show()
  data.plot.scatter("COD_CELLA_CHIAMATA","DATA_CHIAMATA")
  plt.show()

  data.ORA_MIN_CHIAMATA.hist()
  plt.show()
  outputData.info()
  outputData

  pd.to_datetime(outputData.DATA_CHIAMATA, format='%Y%m%d').hist()

  pd.to_datetime(data.DATA_CHIAMATA, format='%Y%m%d').hist()

  data.ORA_MIN_CHIAMATA.hist()
  plt.show()
  outputData.info()
  outputData

  data.ORA_MIN_CHIAMATA.hist()

  inputData=sas[1]
  pd.DataFrame(inputData).COD_CELLA_CHIAMATA.plot()

  data.COD_CELLA_CHIAMATA.value_counts()

#**Aggregated Score Metrics**

In [ ]:
score = evaluate(sd[1], data)